In [3]:
import ROOT as rt
import csv
import re
import sys
import collections
import os
from collections import OrderedDict
import uproot
import pandas as pd

import scipy
import awkward as ak
import numpy as np
import time
import numba
from numba import jit
from matplotlib import pyplot as plt
sys.path.append('/storage/af/user/christiw/login-1/christiw/LLP/Run3/CMSSW_14_0_1/src/mds_analysis/lib/')
# from histo_utilities import create_TH1D, create_TH2D, std_color_list, create_TGraph, make_ratio_plot
from helper_functions import *
# import CMS_lumi, tdrstyle
# tdrstyle.setTDRStyle()

print(sys.version)

Welcome to JupyROOT 6.24/06
3.6.8 (default, Nov  2 2021, 13:01:57) 
[GCC 8.4.1 20200928 (Red Hat 8.4.1-1)]


In [4]:
files = {}

base_path = "/storage/af/user/christiw/login-1/christiw/LLP/Run3/CMSSW_14_0_1/src/mds_analysis_240521/"
# files['mc'] = uproot.open(base_path + 'data/processed/mc_pedro_hlt566_2023.root')['MuonSystem']
# files['data'] = uproot.open(base_path + 'data/processed/r3_pedro_hlt566_2023.root')['MuonSystem']

files['mc'] = uproot.open(base_path + 'data/processed/mc_pedro_hlt566_2023.root')['MuonSystem']
files['data'] = uproot.open(base_path + 'data/processed/r3_pedro_hlt566_2023.root')['MuonSystem']

# NEVENTS = uproot.open(base_path + 'data/raw/mc_2023.root')['NEvents'].values()


In [12]:
%%time

for k,T in files.items():

    # weight = T['weight'].array()* 52*27000/NEVENTS
    if 'data' in k: weight = T['weight'].array()
    else: weight = T['weight'].array()* 1.1328524540090597e-06 * 27.82 * 1000 # from Pedro 2023
    #48/2.31671e+07= 2.0719037e-06?

    presel =  (T["nCscRechitClusters"].array() > 1)
    # cond = T["HLTDecision"].array()[:,566] & (T["nCscRechitClusters"].array() > 1)
    ########### SELECTION: JETS ############

    print("sample", '\t',np.sum(weight[presel]))

    csc = (np.abs(T['gLLP_eta'].array())<3) & (T['gLLP_decay_vertex_r'].array()<800)\
    & (np.abs(T['gLLP_decay_vertex_z'].array())<1200) & (np.abs(T['gLLP_decay_vertex_z'].array())>400)
    # csc = T['cscRechitCluster_match_gLLP'].array() & (np.abs(T['cscRechitCluster_match_gLLP_eta'].array())<3) & (T['cscRechitCluster_match_gLLP_decay_r'].array()<800)\
    # & (np.abs(T['cscRechitCluster_match_gLLP_decay_z'].array())<1200) & (np.abs(T['cscRechitCluster_match_gLLP_decay_z'].array())>400)

    if not 'data' in k:
        presel = presel & (np.sum(csc, axis=1)>1)
        print("acceptance", '\t',np.sum(weight[presel]))

    ## select two clusters per event
    
    presel = presel & (T['nCscRechitClusters'].array()==2) & (T['nDtRechitClusters'].array()==0)
    print("0 DT & 2 CSC clusters", '\t',np.sum(weight[presel]))

    
    # require at least one to pass L1 & time cut
    cscRechitClusterR = (T['cscRechitClusterX'].array()**2+T['cscRechitClusterY'].array()**2)**0.5
    L1 = L1_trg(cscRechitClusterR, np.abs(T['cscRechitClusterZ'].array()), np.abs(T['cscRechitClusterSize'].array()))[presel]
    intime = ((T['cscRechitClusterTimeWeighted'].array() < 12.5) & (T['cscRechitClusterTimeWeighted'].array()> -5) \
    & (T['cscRechitClusterTimeSpreadWeightedAll'].array() < 20))[presel]
    cond =(np.sum(L1 & intime, axis=1)>0)
    weight = weight[presel]
    print("L1 (at least one cluster)", '\t',np.sum(weight[cond]))
    
    # index should be the largest one that passes L1

    cond = cond & (T['met'].array()[presel]<150)
    print("low MET", '\t', np.sum(weight[cond]))

    cond = cond & intime[:,0] & intime[:,1]
    print("intime both clusters", '\t',np.sum(weight[cond]))


    cscRechitClusterPhi = T['cscRechitClusterPhi'].array()[presel]
    cscRechitClusterMuonVetoPt = T['cscRechitClusterMuonVetoPt'].array()[presel]
    cscRechitClusterMuonVetoGlobal = T['cscRechitClusterMuonVetoGlobal'].array()[presel]
    cscRechitClusterJetVetoPt = T['cscRechitClusterJetVetoPt'].array()[presel]
    cscRechitClusterDNN = T['cscRechitClusterDNN_bkgMC_plusBeamHalo'].array()[presel]

    muon_veto = np.logical_not((cscRechitClusterMuonVetoPt > 30) & cscRechitClusterMuonVetoGlobal)
    jet_veto = cscRechitClusterJetVetoPt < 30

    cond = cond & (np.abs(deltaPhi(np.array(cscRechitClusterPhi[:,0]),np.array(cscRechitClusterPhi[:,1]))) >1.8)
    print("dphi < 1.8",'\t', np.sum(weight[cond]))

    cond = cond & (np.sum(muon_veto, axis=1)==2)
    print("muon veto", '\t',np.sum(weight[cond]))

    cond = cond & (np.sum(jet_veto, axis=1)==2)
    print("jet veto", '\t',np.sum(weight[cond]))
    
    
    me1 = (T['cscRechitClusterNRechitChamberPlus11'].array()+ T['cscRechitClusterNRechitChamberPlus12'].array()+\
    T['cscRechitClusterNRechitChamberMinus11'].array()+ T['cscRechitClusterNRechitChamberMinus12'].array()==0)[presel]
    cond = cond & me1[:,0] & me1[:,1]
    print("ME1 veto for both", '\t',np.sum(weight[cond]))


    cond = cond & (np.sum(cscRechitClusterDNN>0.96, axis=1)==2)
    print("DNN",'\t', np.sum(weight[cond]))


sample 	 7791.838
acceptance 	 5868.1523
0 DT & 2 CSC clusters 	 5634.5356
L1 (at least one cluster) 	 4209.476
low MET 	 4065.6562
intime both clusters 	 3867.8125
dphi < 1.8 	 3686.0298
muon veto 	 3662.668
jet veto 	 3074.976
ME1 veto for both 	 1421.4099
DNN 	 1100.9175
sample 	 2550979.0
0 DT & 2 CSC clusters 	 2228811.0
L1 (at least one cluster) 	 1058598.0
low MET 	 1057635.0
intime both clusters 	 70905.0
dphi < 1.8 	 30433.0
muon veto 	 29922.0
jet veto 	 23948.0
ME1 veto for both 	 1021.0
DNN 	 90.0
CPU times: user 7.43 s, sys: 553 ms, total: 7.98 s
Wall time: 7.98 s


In [ ]:
two versions
* one bigger just for cut flow
* one smaller
    * two clusters